<a href="https://colab.research.google.com/github/olaviinha/SloppyButcher/blob/master/amen_beat_reverser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Beat-reverser

Reverses the order of beats of an audio track using [amen](https://github.com/algorithmic-music-exploration/amen).

### Howto:
1. Type or paste something in the `input` field. `input` may be:
  - Path to audio file in Google Drive.
  - Path to directory in Google Drive containing multiple audio files. All audio files in directory will be processed.
  - Youtube-link.
  - Space-separated list of Youtube-links. Each video will be processed.
2. Type a path to directory in your Google Drive where you want output files to be saved.
3. Select <i>Runtime > Run all</i> from the menu.

<font color="#f66"><b>Note:</b></font>
You may ignore the `DeprecationWarning: np.asscalar(a)` errors. Restart runtime after `Setup` cell if you want to get rid of them.
<hr size="1" color="#555"/>
&nbsp;

In [ ]:
#@title Mount Drive

!pip -q install import-ipynb
!curl -s -O https://raw.githubusercontent.com/olaviinha/inhagcutils/master/inhagcutils.ipynb
import import_ipynb
from inhagcutils import *

if not os.path.isdir(drive_root):
  from google.colab import drive
  drive.mount('/content/drive')

In [ ]:
#@title Settings
input = "https://www.youtube.com/watch?v=64j0rPkg_0g" #@param {type:"string"}
output_dir = "incoming/amen" #@param {type:"string"}
output_as = "MP3_192kbps" #@param ["WAV", "MP3_192kbps", "Stem_separated_WAV"]

#@markdown `input`<font size="3" color="#aaa">: path to file in your Drive, path to directory in your Drive containing multiple files, youtube-link or space-separated list of youtube-links.</font><br>
#@markdown `output_dir`<font size="3" color="#aaa">: path to dir where results are saved, relative to your Google Dirve root.</font><br>
#@markdown `output_as`<font size="3" color="#aaa">: `Stem_separated_WAV` = Separate beat-reversed output into four stems via [Spleeter](https://github.com/deezer/spleeter).</font>

output_dir = drive_root+output_dir
dir_tmp = "/content/tmp/"
dir_input = dir_tmp+"input/"
dir_conv = dir_tmp+"converted/"
dir_preview = dir_tmp+"preview/"
dir_results = dir_tmp+"results/"

reset_dirs([dir_tmp, dir_input, dir_conv, dir_preview, dir_results])

input_type = check_input_type(drive_root+input)
if input_type == "file":
  input = drive_root+input
output_dir = fix_path(output_dir)

auto_override = "-y"
format_wav = wav_44
format_mp3 = mp3_192
ffmpeg_verbose = ffmpeg_q
youtube_verbose = youtube_q

### Setup

In [ ]:
#@title Setup
!apt-get install libsndfile1
!pip {pip_q} install librosa --upgrade
!pip {pip_q} install ffmpeg pysoundfile
if input_type == "youtube":
  !pip {pip_q} install youtube-dl
if output_as == "Stem_separated_WAV":
  configSpleeter()
  !pip {pip_q}install spleeter
%cd /content/  
!git clone {git_q} https://github.com/algorithmic-music-exploration/amen.git
%cd /content/amen/

### Beat-reverse

In [ ]:
#@title Process
if input_type == "dir":
  !cp "{input}*" "{dir_input}"
if input_type == "file":
  !cp "{input}" "{dir_input}"
if input_type == "youtube":
  print('Fetching from Youtube...', end='')
  !youtube-dl {youtube_verbose} --restrict-filenames -x --no-continue --audio-format wav -o "{dir_input}%(title)s.%(ext)s" {input}
  print('Done.')

print('Converting...', end='')
sources = list_audio(dir_input)
for source in sources:
  conv_out = dir_conv+basename(source)+'.wav'
  !ffmpeg {ffmpeg_verbose} -i "{source}" {format_wav} "{conv_out}"
print('Done.')

converted = glob(dir_conv+"*.wav")

import amen
#from amen.audio import Audio
from amen.synthesize import synthesize

for audio_file in converted:
  print('\nProcessing', path_leaf(audio_file))
  out_file = dir_results+basename(audio_file)+'.wav'
  print('Analyze...')
  audio = amen.audio.Audio(audio_file)
  beats = audio.timings['beats']
  print('Reverse beats...')
  beats.reverse()
  print('Render...')
  synthesize(beats).output(out_file)
  if output_as == "WAV":
    !cp {out_file} {output_dir}
    out_file_leaf = path_leaf(out_file)
    print('WAV saved to', output_dir+'/'+out_file_leaf)
    print('Encoding preview...')
    preview_out = dir_preview+basename(out_file)+'.mp3'
    !ffmpeg {ffmpeg_verbose} -i "{out_file}" {format_mp3} "{preview_out}"
    audio_player(preview_out)
  elif output_as == "MP3_192kbps":
    print('Encoding...')
    mp3_out = output_dir+basename(out_file)+'.mp3'
    !ffmpeg {ffmpeg_verbose} -i "{out_file}" {format_mp3} "{mp3_out}"
    print('MP3 saved to', mp3_out)
    audio_player(mp3_out)
  elif output_as == "Stem_separated_WAV":
    print('Separating stems...', end='')
    !spleeter separate -i "{out_file}" -o "{output_dir}" -p /content/cfg.json
    print('Done.')
    print('\nPreviews are not provided here when you select output_as Stem_separated_WAV.')